In [1]:
"""
A 1000 run of the gillespied function, with constant inputs 
"""

"A 1000 run of the gillespied function, with constant inputs \n"

In [2]:
using Random, Distributions, Plots, DelimitedFiles

In [3]:
function hazard(x::Vector{Float64}, th::Vector{Float64}, error::Float64)::Vector{Float64}
    k = th[1:5]
    Kc = th[6:7]
    if error>=0
        # deining the k1, & k2 and then subbing them into a vecotr is WAY WAY quicker
        # compared to calcing them in the vector
        k1 = k[1]+error*Kc[1]
        k2 = k[2]+error*Kc[1]
        return [x[1], x[2], x[1], x[2], x[1]].*[k1, k2, k[3], k[4], k[5]]
    else 
        k1 = 2*k[1]/(1+exp(-error*Kc[2]))
        k2 = 2*k[2]/(1+exp(-error*Kc[2]))
        return [x[1], x[2], x[1], x[2], x[1]].*[k1, k2, k[3], k[4], k[5]]
    end
end

hazard (generic function with 1 method)

In [4]:
function hazzy(x::Vector{Float64}, th::Vector{Float64})::Vector{Float64}
    return [x[1], x[2], x[1], x[2], x[1]].*th[1:5]
end

hazzy (generic function with 1 method)

In [ ]:
const post = [[2,0,0,0,1] [0,2,0,0,1]]
const pre = [[1,0,1,0,1] [0,1,0,1,0]]
const S = post - pre
const k = [3.06e-8, 3.06e-8, 3.06e-8, 3.06e-8, 0.0, 8.99e-9, 2e-3];

In [8]:
function gen_inits(μ::Real, σ::Real, α::Real, β::Real)::Vector{Float64}
    CC = rand(Normal(μ, σ))
    hh = rand(Beta(α, β))
    return round.( [CC*(1-hh), CC*hh] )
end

"""
When specifying the parameters as ::Float64 the function is consistently a slower
...weird but okay.
"""

"When specifying the parameters as ::Float64 the function is consistently a slower\n...weird but okay.\n"

In [21]:
function gillespied(inits::Vector{Float64}, k::Vector{Float64}, S::Matrix{Int64}, Tmax::Int64, δt::Int64)
    x = inits
    tt = 0.0
    n = trunc(Int, Tmax/δt)
    xmat = fill(-1, (2, n))
    i = 1
    target = 0.0
    C0 = sum(x)
    while i <= n
        error = C0 - sum(x)
        h = hazzy(x, k)
        h0 = sum(h)
        if h0<1e-10
            xmat[:,i:n] = fill(0.0, (2,n-i+1))
            return xmat'
        else
            Exp = Exponential(1/h0)
            tt = tt + rand(Exp)
        end
        while tt>=target && i<=n
            xmat[:,i] = x
            i += 1
            target += δt
        end
        Cat = Categorical(h/h0)
        r = rand(Cat)
        x += S'[:,r]
    end
    return xmat'
end

gillespied (generic function with 1 method)

In [10]:
function replace_nan(x)
    for i=eachindex(x)
        x[i] = isnan(x[i]) ? 0.0 : x[i]
    end
end

replace_nan (generic function with 1 method)

In [11]:
function raw_to_summ(sims)::Array{Float64}
    """
    converts the species populations from the gillespie algorithm to 
    copy number and mutation load
    """
    Nsim = size(sims)[3] # no. of simulations
    n = size(sims)[1] # length of one simulation
    out = Array{Float64}(undef, n,2,Nsim)
    for i=1:Nsim
        copy_num = sum(sims[:,:,i], dims=2)
        mut_load = sims[:,2,i]./copy_num
        replace_nan(copy_num)
        replace_nan(mut_load)
        out[:,:,i] = hcat(copy_num, mut_load)
    end
    out
end

raw_to_summ (generic function with 1 method)

In [12]:
function quantiles(sims, p)
    """
    returns quantile summaries from simulations
    """
    Nsim = size(sims)[3] # Nsim: number of simulations
    n = size(sims)[1] # length of one simulation
    out = Array{Float64}(undef, n,length(p),2)
    for t=1:n
        out[t,:,1] = quantile([sims[t,1,i] for i=1:Nsim], p)
        out[t,:,2] = quantile([sims[t,2,i] for i=1:Nsim], p)
    end
    out
end

quantiles (generic function with 1 method)

In [13]:
Nsim = 1000
Tmax = 80*365*24*3600
δt = 24*3600;

In [14]:
@time [ gillespied([100.0,100.0], k, S, Tmax, δt) for i=1:Nsim ];

 11.348419 seconds (219.81 M allocations: 24.709 GiB, 9.92% gc time, 2.30% compilation time)


In [15]:
simulations = Array{Float64}(undef, trunc(Int, Tmax/δt), 2, Nsim)
for i=1:Nsim
    simulations[:,:,i] = gillespied([100.0,100.0], k ,S, Tmax, δt)
end

In [16]:
CH_sim = raw_to_summ(simulations);

In [17]:
quant_sim = quantiles(CH_sim, [0.025,0.1,0.5,0.9,0.975]) ;